17.Les requêtes SQL Via SQLAlchemy

-Insérer les données nettoyées dans une table PostgreSQL nommée, par exemple : -health_data_cleaned.
-Quelle est la répartition des individus par genre (gender)
-Quelle est la répartition des individus en fonction de leurs habitudes de tabagisme (smoking) ?
-Quelle est la moyenne de l'IMC (bmi) pour chaque genre (gender) ?
-Comment les individus se répartissent-ils en fonction de leur niveau d'éducation (education) ?
-Quelle est l'évolution de l'IMC moyen (bmi) en fonction des tranches d'âge (par exemple, 18-30, 31-50, 51+) ?
-Quelle est la moyenne d'âge (age) pour chaque catégorie de tabagisme (smoking) ?

In [ ]:
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import pandas as pd
import os

In [2]:


load_dotenv()

DATABASE_URL = os.getenv("DATABASE_URL")

engine = create_engine(DATABASE_URL, echo=True)

df_cleaned = pd.read_csv('data_final.csv')

df_cleaned.to_sql('health_data_cleaned', con=engine, if_exists='replace', index=False)

print("✅ Données insérées avec succès dans la table 'health_data_cleaned'")


2025-07-10 22:17:40,019 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-07-10 22:17:40,019 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-07-10 22:17:40,019 INFO sqlalchemy.engine.Engine select current_schema()
2025-07-10 22:17:40,019 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-07-10 22:17:40,019 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-07-10 22:17:40,035 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-07-10 22:17:40,035 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-10 22:17:40,056 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

In [3]:
from sqlalchemy import text

query_gender = text("SELECT gender, COUNT(*) FROM health_data_cleaned GROUP BY gender")
with engine.connect() as conn:
    result = conn.execute(query_gender).fetchall()
    print("Répartition par genre:", result)

print("************************************************************")



2025-07-10 22:17:41,217 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-10 22:17:41,224 INFO sqlalchemy.engine.Engine SELECT gender, COUNT(*) FROM health_data_cleaned GROUP BY gender
2025-07-10 22:17:41,229 INFO sqlalchemy.engine.Engine [generated in 0.00718s] {}
Répartition par genre: [('female', 2498), ('male', 2346)]
2025-07-10 22:17:41,239 INFO sqlalchemy.engine.Engine ROLLBACK
************************************************************


In [4]:
query_smoking = text("SELECT smoking, COUNT(*) FROM health_data_cleaned GROUP BY smoking")
with engine.connect() as conn:
    print("Répartition par tabagisme:", conn.execute(query_smoking).fetchall())

print("************************************************************")

2025-07-10 22:17:41,268 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-10 22:17:41,284 INFO sqlalchemy.engine.Engine SELECT smoking, COUNT(*) FROM health_data_cleaned GROUP BY smoking
2025-07-10 22:17:41,284 INFO sqlalchemy.engine.Engine [generated in 0.00589s] {}
Répartition par tabagisme: [('yes', 1991), ('no', 2853)]
2025-07-10 22:17:41,300 INFO sqlalchemy.engine.Engine ROLLBACK
************************************************************


In [5]:

query_bmi_gender = text("SELECT gender, AVG(bmi) FROM health_data_cleaned GROUP BY gender")
with engine.connect() as conn:
    print("Moyenne BMI par genre:", conn.execute(query_bmi_gender).fetchall())

print("************************************************************")

2025-07-10 22:17:41,339 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-10 22:17:41,346 INFO sqlalchemy.engine.Engine SELECT gender, AVG(bmi) FROM health_data_cleaned GROUP BY gender
2025-07-10 22:17:41,347 INFO sqlalchemy.engine.Engine [generated in 0.00678s] {}
Moyenne BMI par genre: [('female', 28.81834240263993), ('male', 28.074151362201945)]
2025-07-10 22:17:41,359 INFO sqlalchemy.engine.Engine ROLLBACK
************************************************************


In [6]:
query_edu = text("SELECT education, COUNT(*) FROM health_data_cleaned GROUP BY education")
with engine.connect() as conn:
    print("Répartition par éducation:", conn.execute(query_edu).fetchall())

print("************************************************************")

2025-07-10 22:17:41,400 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-10 22:17:41,400 INFO sqlalchemy.engine.Engine SELECT education, COUNT(*) FROM health_data_cleaned GROUP BY education
2025-07-10 22:17:41,411 INFO sqlalchemy.engine.Engine [generated in 0.00856s] {}
Répartition par éducation: [('College or above', 1319), ('HS or GED', 1132), ('9-11th grade', 613), ('Some college / AA', 1780)]
2025-07-10 22:17:41,416 INFO sqlalchemy.engine.Engine ROLLBACK
************************************************************


In [ ]:
query_bmi_age_group = text("""
SELECT 
  CASE 
    WHEN age BETWEEN 18 AND 30 THEN '18-30'
    WHEN age BETWEEN 31 AND 50 THEN '31-50'
    ELSE '51+'
  END AS age_group,
  AVG(bmi)
FROM health_data_cleaned
GROUP BY age_group
""")

print("************************************************************")

with engine.connect() as conn:
    print("BMI moyen par tranche d’âge:", conn.execute(query_bmi_age_group).fetchall())

print("************************************************************")


************************************************************
2025-07-10 22:17:41,453 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-10 22:17:41,453 INFO sqlalchemy.engine.Engine 
SELECT 
  CASE 
    WHEN age BETWEEN 18 AND 30 THEN '18-30'
    WHEN age BETWEEN 31 AND 50 THEN '31-50'
    ELSE '51+'
  END AS age_group,
  AVG(bmi)
FROM health_data_cleaned
GROUP BY age_group

2025-07-10 22:17:41,453 INFO sqlalchemy.engine.Engine [generated in 0.00561s] {}
BMI moyen par tranche d’âge: [('18-30', 26.841494866311244), ('51+', 28.862636516816455), ('31-50', 29.15740735441536)]
2025-07-10 22:17:41,469 INFO sqlalchemy.engine.Engine ROLLBACK
************************************************************


In [8]:
query_age_smoking = text("SELECT smoking, AVG(age) FROM health_data_cleaned GROUP BY smoking")
with engine.connect() as conn:
    print("Âge moyen par catégorie de tabagisme:", conn.execute(query_age_smoking).fetchall())

2025-07-10 22:17:41,525 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-10 22:17:41,525 INFO sqlalchemy.engine.Engine SELECT smoking, AVG(age) FROM health_data_cleaned GROUP BY smoking
2025-07-10 22:17:41,533 INFO sqlalchemy.engine.Engine [generated in 0.00582s] {}
Âge moyen par catégorie de tabagisme: [('yes', Decimal('51.6167754897036665')), ('no', Decimal('43.7690150718541886'))]
2025-07-10 22:17:41,546 INFO sqlalchemy.engine.Engine ROLLBACK
